In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

#view all all columns for large df
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline
import seaborn as sns

In [ ]:
cv1 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/fcs-crisis-data-campers/data/NSSCrisisv.1.xlsx')
cv1_df= pd.DataFrame(cv1)

In [ ]:
cv1_df

In [ ]:
cv1_df.info()

In [ ]:
cv1_df.isnull().sum()

In [ ]:
cv2 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/fcs-crisis-data-campers/data/NSSCrisisv.2.xlsx')
cv2_df= pd.DataFrame(cv2)

In [ ]:
cv2_df

In [ ]:
cv2_df.info()

In [ ]:
cv2_df.isnull().sum()

In [ ]:
labels = list()
for col in cv2_df.columns:
    labels.append(col.replace('Subjective', 'Issues'))
cv2_df.columns = labels

In [ ]:
filter_col = [col for col in cv1_df if col.startswith('CRISIS Issues') or col.startswith('Call')]
cv1_df_sub = cv1_df[filter_col]
cv1_df_sub

In [ ]:
filter_col = [col for col in cv2_df if col.startswith('CRISIS Issues') or col.startswith('Call')]
cv2_df_sub = cv2_df[filter_col]
cv2_df_sub

In [ ]:
frames = [cv1_df_sub,cv2_df_sub]
df_combined = pd.concat(frames)

In [ ]:
df_combined 

In [ ]:
df_combined.drop(['CallersFeedback'], axis=1)

In [ ]:
df_combined.isnull().sum()

In [ ]:
df_combined['Year'] = pd.DatetimeIndex(df_combined['CallDateAndTimeStart']).year
df_combined['Month_Number'] = pd.DatetimeIndex(df_combined['CallDateAndTimeStart']).month
df_combined['Month_Name'] = pd.DatetimeIndex(df_combined['CallDateAndTimeStart']).month_name()
df_combined['Day'] = pd.DatetimeIndex(df_combined['CallDateAndTimeStart']).day
df_combined['Day_Number'] = pd.DatetimeIndex(df_combined['CallDateAndTimeStart']).day_of_week
df_combined['Day_Name'] = pd.DatetimeIndex(df_combined['CallDateAndTimeStart']).day_name()
df_combined

In [ ]:
df_combined.info()

## Rob's code to count speccific values in the column

In [ ]:
mental_health_df = df_combined['CRISIS Issues - Mental Health'].notnull()

In [ ]:
mental_health_df = df_combined[mental_health_df]

In [ ]:
count = []

for x in mental_health_df['CRISIS Issues - Mental Health']:
    counter0 = x.count(";") + 1   
    count.append(counter0)


In [ ]:
mental_health_df['MH_Issues_Count'] = count

In [ ]:
mental_health_df

In [ ]:
crisis_by_caller = df_combined.drop(['CallDateAndTimeStart','CallDateAndTimeEnd','CallLength','CallerNum','CallersFeedback','Day_Number','Day','Day_Name'], axis=1)
crisis_by_caller

In [ ]:
crisis_by_caller_new = crisis_by_caller.groupby(["CallReportNum","Year","Month_Name","Month_Number"]).count().reset_index()
crisis_by_caller_new

In [ ]:
crisis_by_caller_new.info()

In [ ]:
crisis_by_caller_new["CallReportNum"] = crisis_by_caller_new["CallReportNum"].astype(str)
crisis_by_caller_new["Year"] = crisis_by_caller_new["Year"].astype(str)
crisis_by_caller_new["Month_Number"] = crisis_by_caller_new["Month_Number"].astype(str)

In [ ]:
crisis_by_caller_new.info()

In [ ]:
crisis_by_caller_new['Totals'] = crisis_by_caller_new.sum(axis=1)
crisis_by_caller_new

In [ ]:
crisis_by_month = crisis_by_caller_new.groupby(["Year","Month_Name","Month_Number"]).agg({"CallReportNum":'count', "Totals": 'sum'}).reset_index()
crisis_by_month

In [ ]:
crisis_by_month.info()

### Average Number of Issues per call based on the month

In [ ]:
crisis_by_month["Average"] = crisis_by_month ["Totals"] / crisis_by_month ["CallReportNum"]
crisis_by_month

In [ ]:
crisis_by_caller2 = df_combined.drop(['CallDateAndTimeStart','CallDateAndTimeEnd','CallLength','CallerNum','CallersFeedback','Day_Number','Day'], axis=1)
crisis_by_caller2

In [ ]:
crisis_by_caller_new2 = crisis_by_caller2.groupby(["CallReportNum","Year","Month_Name","Day_Name","Month_Number"]).count().reset_index()
crisis_by_caller_new2

In [ ]:
crisis_by_caller_new2.info()

In [ ]:
crisis_by_caller_new2["CallReportNum"] = crisis_by_caller_new2["CallReportNum"].astype(str)
crisis_by_caller_new2["Year"] = crisis_by_caller_new2["Year"].astype(str)
crisis_by_caller_new2["Month_Number"] = crisis_by_caller_new2["Month_Number"].astype(str)

In [ ]:
crisis_by_caller_new2['Totals'] = crisis_by_caller_new2.sum(axis=1)
crisis_by_caller_new2

In [ ]:
crisis_by_day = crisis_by_caller_new2.groupby(["Year","Month_Name","Day_Name","Month_Number"]).agg({"CallReportNum":'count', "Totals": 'sum'}).reset_index()
crisis_by_day

### Average Number of Issues per call based on the day of the week

In [ ]:
crisis_by_day["Average"] = crisis_by_day["Totals"] / crisis_by_day ["CallReportNum"]
crisis_by_day

In [ ]:
plt.bar("Month_Name","Average", align='center', alpha=0.5)